# Part 1 Preprocess
* * *

In [277]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox

from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import  StandardScaler

from statsmodels.genmod.generalized_linear_model import GLM
import statsmodels.api as sm
from scipy import stats


In [278]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


### Loading the datasets

Create a list of all files and add the corresponding city names and day type

In [279]:
def load_data():
    data = [("/Users/lauraforerocamacho/Downloads/data/amsterdam_weekdays.csv", "Amsterdam", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/athens_weekdays.csv", "Athens","Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/barcelona_weekdays.csv", "Barcelona", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/berlin_weekdays.csv", "Berlin", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/budapest_weekdays.csv", "Budapest", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/lisbon_weekdays.csv", "Lisbon", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/London_weekdays.csv", "London", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/rome_weekdays.csv", "Rome", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/paris_weekdays.csv", "Paris", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/Vienna_weekdays.csv", "Vienna", "Weekdays"),
                 ("/Users/lauraforerocamacho/Downloads/data/amsterdam_weekends.csv", "Amsterdam", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/athens_weekends.csv", "Athens", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/barcelona_weekends.csv", "Barcelona", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/berlin_weekends.csv", "Berlin", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/budapest_weekends.csv", "Budapest", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/lisbon_weekends.csv", "Lisbon", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/London_weekends.csv", "London", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/rome_weekends.csv", "Rome", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/paris_weekends.csv", "Paris", "Weekends"),
                 ("/Users/lauraforerocamacho/Downloads/data/Vienna_weekends.csv", "Vienna", "Weekends")
                 ]
    #merge data
    dfs = []
    for file_path, city_name, day_type in data:
        df = pd.read_csv(file_path, usecols=lambda col: col != 'Unnamed: 0')
        df = df.assign(city=city_name, type=day_type)
        dfs.append(df)
    data = pd.concat(dfs)
    return data

data= load_data()
numeric_colums= ["realSum", "person_capacity","cleanliness_rating","guest_satisfaction_overall",'bedrooms', 'dist',
       'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm', 'lng', 'lat']
categorical_colums= ["room_type","room_shared", "room_private", "host_is_superhost","multi", "biz" , 'city', 'type']

def preprocess_data(data):
    for i in categorical_colums:
        data[i] = data[i].astype('object')
        data[i].unique()
    return data
data= preprocess_data(data)

View distributin of the value

In [280]:
import seaborn as sns

def plot_distribution_features():

    fig, axes = plt.subplots(6,4,figsize=(26,20))

    # We will not plot country because it has too many categories.
    for i, c in enumerate(data.columns):
        ax = axes.reshape(-1)[i]
        if data[c].dtype.kind == 'O':
            a = sns.countplot(x=c,data=data,ax=ax)
        else:
            b = sns.histplot(x=c,data=data,ax=ax)
        t = ax.set_title(c)
    plt.tight_layout()

In [281]:
data_clean=data

def preprocess_data_LL(data_clean):
    for i in  ['Amsterdam','Athens','Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris', 'Rome','Vienna']:
        data_clean["lat_"+i]=0
        data_clean["lng_"+i]=0
    data_clean.reset_index()
    for ind in data_clean.index:
        data_clean['lat_'+data_clean['city'][ind]][ind]=data_clean['lat'][ind]
        data_clean['lng_'+data_clean['city'][ind]][ind]=data_clean['lng'][ind]
    data_clean.drop(columns=['lng','lat'],inplace=True)
    return data_clean
#data_clean = preprocess_data_LL(data_clean)
X = data_clean.loc[:,data_clean.columns != 'realSum']
y = data_clean['realSum']


In [282]:
for i in ["room_shared",	"room_private",	"host_is_superhost"]:
    X[i].replace([False, True],[0,1],inplace=True)

#X['person_capacity'] = X['person_capacity'].astype(str)
#X['bedrooms'] = X['bedrooms'].astype(str)
#,'person_capacity'
#"room_type",
for column in [ "city", "type"]:
    X_one_hot = pd.get_dummies(X[column] , dtype=int, drop_first=False)
    X = pd.concat([X, X_one_hot], axis='columns')

# drop the values
X= X.drop(["room_type", "city", "type"], axis='columns')
X_new= X


/var/folders/zv/hzjbp8kd1l103x8tgl0s00z80000gn/T/ipykernel_3968/116871885.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i].replace([False, True],[0,1],inplace=True)


In [283]:
final_train_data=X_new
final_train_data["realSum"]=y

In [284]:
final_train_data.to_csv("datapre.csv", index=False)

In [285]:
from pysal.model import spreg
from pysal.lib import weights
from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns


In [286]:
import geopandas as gpd
from pysal.model import spreg
from pysal.lib import weights
from scipy import stats
# Create a GeoDataFrame
final_train_data["geometry"] = gpd.points_from_xy(final_train_data["lng"], data["lat"])
final_train_data = gpd.GeoDataFrame(final_train_data, crs="epsg:4326")

final_train_data = final_train_data.reset_index(drop=True)

# Check the first rows
final_train_data.head()

,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,...,Budapest,Lisbon,London,Paris,Rome,Vienna,Weekdays,Weekends,realSum,geometry
0,0,1,2.0,0,1,0,10.0,93.0,1,5.022964,...,0,0,0,0,0,0,1,0,194.033698,POINT (4.90569 52.41772)
1,0,1,4.0,0,0,0,8.0,85.0,1,0.488389,...,0,0,0,0,0,0,1,0,344.245776,POINT (4.90005 52.37432)
2,0,1,2.0,0,0,1,9.0,87.0,1,5.748312,...,0,0,0,0,0,0,1,0,264.101422,POINT (4.97512 52.36103)
3,0,1,4.0,0,0,1,9.0,90.0,2,0.384862,...,0,0,0,0,0,0,1,0,433.529398,POINT (4.89417 52.37663)
4,0,1,2.0,1,0,0,10.0,98.0,1,0.544738,...,0,0,0,0,0,0,1,0,485.552926,POINT (4.90051 52.37508)


In [287]:
final_train_data["log_price"] = np.log(final_train_data["realSum"] + 0.000001)

In [288]:
final_train_data[['multi', 'biz']]=final_train_data[['multi', 'biz']].astype('float64')


In [289]:
train_data, test_data = train_test_split(final_train_data, test_size=0.2, random_state=42,
                                                    stratify=final_train_data[[           
        'room_shared', 'room_private' , 'multi']])

In [290]:
w = weights.KNN.from_dataframe(train_data, k=8)
w.transform = 'R'
w

/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 28 disconnected components.
  warnings.warn(message)


In [291]:
X_new.columns

Index(['room_shared', 'room_private', 'person_capacity', 'host_is_superhost',
       'multi', 'biz', 'cleanliness_rating', 'guest_satisfaction_overall',
       'bedrooms', 'dist', 'metro_dist', 'attr_index', 'attr_index_norm',
       'rest_index', 'rest_index_norm', 'lng', 'lat', 'Amsterdam', 'Athens',
       'Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris', 'Rome',
       'Vienna', 'Weekdays', 'Weekends', 'realSum', 'geometry'],
      dtype='object')

In [292]:
train_data.dtypes

room_shared                      int64
room_private                     int64
person_capacity                float64
host_is_superhost                int64
multi                          float64
biz                            float64
cleanliness_rating             float64
guest_satisfaction_overall     float64
bedrooms                         int64
dist                           float64
metro_dist                     float64
attr_index                     float64
attr_index_norm                float64
rest_index                     float64
rest_index_norm                float64
lng                            float64
lat                            float64
Amsterdam                        int64
Athens                           int64
Barcelona                        int64
Berlin                           int64
Budapest                         int64
Lisbon                           int64
London                           int64
Paris                            int64
Rome                     

In [293]:


features = ['room_shared', 'room_private', 'person_capacity', 'host_is_superhost',
       'cleanliness_rating', 'guest_satisfaction_overall','multi', 'biz',
       'bedrooms', 'dist', 'metro_dist', 'attr_index_norm',
       'rest_index_norm', 'Weekdays', 'Athens',
       'Barcelona', 'Berlin', 'Budapest', 'Lisbon', 'London', 'Paris', 'Rome',
       'Vienna']
target = 'realSum'


In [294]:
all_model_attributes = ["realSum"] + features
has_nans = False
for attr in all_model_attributes:
    if train_data[attr].hasnans:
        has_nans = True
print("Has missing values:", has_nans)

Has missing values: False


In [295]:
train_data = train_data.dropna(subset=all_model_attributes).copy()


In [296]:
m1 = spreg.OLS(train_data[['realSum']].values, train_data[features].values, 
                  name_y = 'realSum', name_x = features)

results = m1

print(results.summary)


REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     realSum                Number of Observations:       41365
Mean dependent var  :    279.5657                Number of Variables   :          24
S.D. dependent var  :    339.3661                Degrees of Freedom    :       41341
R-squared           :      0.2247
Adjusted R-squared  :      0.2242
Sum squared residual:3693617003.718                F-statistic           :    520.8172
Sigma-square        :   89345.130                Prob(F-statistic)     :           0
S.E. of regression  :     298.907                Log likelihood        : -294468.308
Sigma-square ML     :   89293.292                Akaike info criterion :  588984.616
S.E of regression ML:    298.8198                Schwarz criterion     :  589191.741

---------------------------------------------------------------------------

In [297]:
m2 = spreg.OLS(train_data[['log_price']].values, train_data[features].values, 
                  name_y = 'log_price', name_x = features)
print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :   log_price                Number of Observations:       41365
Mean dependent var  :      5.4175                Number of Variables   :          24
S.D. dependent var  :      0.5956                Degrees of Freedom    :       41341
R-squared           :      0.6569
Adjusted R-squared  :      0.6567
Sum squared residual:    5034.411                F-statistic           :   3441.9695
Sigma-square        :       0.122                Prob(F-statistic)     :           0
S.E. of regression  :       0.349                Log likelihood        :  -15134.181
Sigma-square ML     :       0.122                Akaike info criterion :   30316.363
S.E of regression ML:      0.3489                Schwarz criterion     :   30523.487

-----------------------------------------------------------------------------

In [298]:
from sklearn.metrics import mean_absolute_error

m1.intercept = m1.betas[0]  # Get the intercept from the betas array
m1.coefficients = m1.betas[1:len(m1.betas)] # Get the coefficients from the betas array
predicted_price = m1.intercept + test_data[features].dot(  m1.coefficients) 
predicted_price_2 = m1.intercept + train_data[features].dot(  m1.coefficients) 

# Calculate MSE for different models
mse_ols_1 = mean_absolute_error(train_data["realSum"], predicted_price_2)
mse_ols = mean_absolute_error(test_data["realSum"], predicted_price)


print("Normal model")

print(mse_ols_1)
print(mse_ols)



m2.intercept = m2.betas[0]  # Get the intercept from the betas array
m2.coefficients = m2.betas[1:len(m2.betas)] # Get the coefficients from the betas array
predicted_price = m2.intercept + test_data[features].dot(  m2.coefficients) 
predicted_price_2 = m2.intercept + train_data[features].dot(  m2.coefficients) 

# Calculate MSE for different models
mse_ols_1 = mean_absolute_error(train_data["log_price"], predicted_price_2)
mse_ols = mean_absolute_error(test_data["log_price"], predicted_price)

print("Log model")

print(mse_ols_1)
print(mse_ols)

predicted_price = np.exp(predicted_price) - 0.000001 
predicted_price_2 = np.exp(predicted_price_2) - 0.000001 


# Calculate MSE for different models
mse_ols_1 = mean_absolute_error(train_data["realSum"], predicted_price_2)
mse_ols = mean_absolute_error(test_data["realSum"], predicted_price)

print("Log model-after")

print(mse_ols_1)
print(mse_ols)



Normal model
99.38673353561202
97.8410051910782
Log model
0.2577193799516518
0.257707519329221
Log model-after
79.870526567546
78.58566870737724


In [299]:
w_pool = weights.KNN.from_dataframe(train_data, k=8)
lagged = train_data.assign(w_pool=weights.spatial_lag.lag_spatial(w_pool, train_data['rest_index'].values))


/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 28 disconnected components.
  warnings.warn(message)


In [300]:
extended_vars = features
extended_vars.append("w_pool")
train_data['w_pool']=lagged['w_pool']
m1 = spreg.OLS(train_data[['realSum']].values, train_data[extended_vars].values,
                  name_y = 'realSum', name_x = extended_vars)

results = m1

print(results.summary)


REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     realSum                Number of Observations:       41365
Mean dependent var  :    279.5657                Number of Variables   :          25
S.D. dependent var  :    339.3661                Degrees of Freedom    :       41340
R-squared           :      0.2247
Adjusted R-squared  :      0.2243
Sum squared residual:3693416797.014                F-statistic           :    499.2249
Sigma-square        :   89342.448                Prob(F-statistic)     :           0
S.E. of regression  :     298.902                Log likelihood        : -294467.187
Sigma-square ML     :   89288.452                Akaike info criterion :  588984.374
S.E of regression ML:    298.8117                Schwarz criterion     :  589200.129

---------------------------------------------------------------------------

In [301]:
w_pool = weights.KNN.from_dataframe(test_data, k=8)
lagged = test_data.assign(w_pool=weights.spatial_lag.lag_spatial(w_pool, test_data['rest_index'].values))
test_data['w_pool']=lagged['w_pool']


/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 11 disconnected components.
  warnings.warn(message)


In [302]:
from sklearn.metrics import mean_absolute_error

m1.intercept = m1.betas[0]  # Get the intercept from the betas array
m1.coefficients = m1.betas[1:len(m1.betas)] # Get the coefficients from the betas array
predicted_price = m1.intercept + test_data[extended_vars].dot( m1.coefficients) 
predicted_price_2 = m1.intercept + train_data[extended_vars].dot( m1.coefficients) 

# Calculate MSE for different models
mse_ols_1 = mean_absolute_error(train_data["realSum"], predicted_price_2)
mse_ols = mean_absolute_error(test_data["realSum"], predicted_price)

print(mse_ols_1)
print(mse_ols)

99.37970718062525
97.85185798337572
